In [63]:
#Package installations for GoogleColab
!pip install deeplake
!pip install "deeplake[enterprise]"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [64]:
import deeplake

In [65]:
ds = deeplake.load('hub://activeloop/fgnet')

train_ds, val_ds = ds.random_split([0.8, 0.2])

print(len(train_ds), len(val_ds))

hub://activeloop/fgnet loaded successfully.

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/fgnet

802 200


In [66]:
dataloader = ds.pytorch(num_workers=0, batch_size=4, shuffle=False)

In [67]:
dataloader = ds.tensorflow()

In [68]:
ds.summary()

Dataset(path='hub://activeloop/fgnet', read_only=True, tensors=['images', 'keypoints', 'ages', 'image_ids'])

  tensor        htype                   shape               dtype  compression
  -------      -------                 -------             -------  ------- 
  images        image       (1002, 359:772, 300:639, 1:3)   uint8    jpeg   
 keypoints  keypoints_coco         (1002, 204, 1)           int32    None   
   ages      class_label              (1002, 1)            uint32    None   
 image_ids   class_label              (1002, 1)            uint32    None   


In [69]:
ds.visualize()

In [70]:
#Strictly observational

image = ds.images[0].numpy() # Fetch the first image and return a numpy array
labels = ds.ages[0].data() # Fetch the labels in the first image

# Slicing
img_list = ds.ages[0:100].numpy(aslist=True) # Fetch 100 labels and store 
                                               # them as a list of numpy arrays



Building A sample Neural Net

In [72]:
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

In [80]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.ToPILImage(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

# Since torchvision transforms expect PIL images, we use the 'pil' decode_method for the 'images' tensor. This is much faster than running ToPILImage inside the transform
trainloader = train_ds.pytorch(transform = {'images': transform, 'ages': None}, batch_size = 1, decode_method = {'images': 'pil'})
testloader = val_ds.pytorch( transform = {'images': transform, 'ages': None}, batch_size = 1, decode_method = {'images': 'pil'})

In [76]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [77]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [78]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

TransformFailedError: ignored